In [5]:
#!pip install pql
import pql
testAll = pql.find("elemets == all(['Si', 'O', 3])")

In [6]:
from optimade.filter import Parser
from lark import Transformer
p = Parser(version=(0, 9, 5))

In [191]:
import pql
from optimade.filter import Parser
from lark import Transformer


optiMadeToPQLOperatorSwitch = {
    "=":"==",
    "<=":"<=",
    ">=":">=",
    "=>":">=",
    "!=": "!=",
    "<":"<",
    ">":">",
}

def OptiMadeToPQLOperatorValidator(x):
    return optiMadeToPQLOperatorSwitch[x]

def combineMultiple(string, index):
    firstQuoteIndex = 0
    for firstQuoteIndex in reversed(range(0, index)):
        if(ord(string[firstQuoteIndex]) == 34):
            break
    lastQuoteIndex = index
    for lastQuoteIndex in range(index, len(string)):
        if(ord(string[lastQuoteIndex])== 34):
            break
    insertion = string[firstQuoteIndex+1:lastQuoteIndex]
    insertion = insertion.split(",")
    return string[:firstQuoteIndex] + 'all({})'.format(insertion) + string[lastQuoteIndex+1:]

def parseInput(PQL):
    length = len(PQL)
    i = 0
    while(i < length):
        if(PQL[i] == ","):
            PQL = combineMultiple(PQL, i)
        i = i + 1 
    return PQL

class OptimadeToPQLTransformer(Transformer):
    def comparison(self, args):
        A = str(args[0])
        B = str(args[2])
        operator = OptiMadeToPQLOperatorValidator(args[1])
#         print('"' + A +'"' + operator + '"' + B + '"')
        return '"' + A +'"' + operator + '"' + B + '"'
    def atom(self, args):
        return args[0]
    def term(self, args):
        result = ""
        for arg in args:
            result = result + " " + arg
        return result.lstrip()
    def expression(self, args):
        result = ""
        for arg in args:
            result = result + " " + arg
        return result.lstrip()
    def start(self, args):
        return args[1]

def optimadeToMongoDBConverter(optimadeQuery):
    p = Parser(version=(0, 9, 5))
#     print("optimadeQuery:", optimadeQuery)
#     print()
    tree = p.parse(optimadeQuery)
#     print("tree:\n", tree)
#     print()
    rawPQL = OptimadeToPQLTransformer().transform(tree)
    cleanedPQL = parseInput(rawPQL)
#     print(cleanedPQL)
#     print()
    try:
        return pql.find(cleanedPQL)
    except:
        return "ERROR: Unable to execute conversion"
    

In [197]:
# sampleInput1 = 'filter=formula_prototype="AB2C3" and nelements=4 and elements="Si, O"'
# sampleInput2 = "filter=formula_prototype='AB2C3' or nelements=4 or elements='Si, O'"
# sampleInput3 = 'filter=formula_prototype="AB2C3" and nelements=4 or elements="Si, O"'
# sampleInput4 = 'filter=formula_prototype="AB2C3" and (nelements=4 or elements="Si, O")'
# print("input1:", optimadeToMongoDBConverter(sampleInput1))
# print("result:\n", optimadeToMongoDBConverter(sampleInput2))
# x = optimadeToMongoDBConverter(sampleInput3)
x = optimadeToMongoDBConverter('filter=a<0 and b>+2')

In [201]:
pql.find("a < '-1'")

{'a': {'$lt': '-1'}}

In [184]:
import unittest
class ConverterTest(unittest.TestCase):
    def setup(self):
        pass
    def testAll(self):
        self.test_one_input()
        self.test_two_inputs_with_and()
        self.test_two_inputs_with_or()
        self.test_valid_numbers_positive()
        
    def test_one_input(self):
        testCase = "filter=a<0" 
        answer = {'a': {'$lt': 0}}
        message = "test_one_input failed"
        self.assertEqual(optimadeToMongoDBConverter(testCase), answer, message)
        print("passed: test_one_input passed")
    def test_two_inputs_with_and(self):
        testCase = "filter=a<0 and b>2" 
        answer = {'$and': [{'a': {'$lt': 0}}, {'b': {'$gt': 2}}]}
        message = "test_two_inputs_with_and failed"
        self.assertEqual(optimadeToMongoDBConverter(testCase), answer, message)
        print("passed: test_two_inputs_with_and")
    def test_two_inputs_with_or(self):
        testCase = "filter=a<0 or b>2" 
        answer = {'$or': [{'a': {'$lt': 0}}, {'b': {'$gt': 2}}]}
        message = "test_two_inputs_with_or failed"
        self.assertEqual(optimadeToMongoDBConverter(testCase), answer, message)
        print("passed: test_two_inputs_with_or")
    def test_valid_numbers_with_positive_sign(self):
        testCase = "filter= a=+1" 
        answer = {'a': +1}
        message = "test_valid_numbers_positive failed"
        self.assertEqual(optimadeToMongoDBConverter(testCase), answer, message)
        print("passed: test_valid_numbers_positive")

In [185]:
ConverterTest().testAll()

"a"<"0"
"a"<"0"



AssertionError: {'a': {'$lt': '0'}} != {'a': {'$lt': 0}}
- {'a': {'$lt': '0'}}
?               - -

+ {'a': {'$lt': 0}} : test_one_input failed